In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds

In [3]:
imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

In [22]:
train_data = imdb['train']
test_data = imdb['test']

In [10]:
print(dir(train_data))

['_GeneratorState', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_add_variable_with_custom_getter', '_apply_options', '_as_serialized_graph', '_as_variant_tensor', '_checkpoint_dependencies', '_component_metadata', '_consumers', '_dataset', '_deferred_dependencies', '_flat_shapes', '_flat_structure', '_flat_types', '_from_components', '_functions', '_gather_saveables_for_checkpoint', '_graph', '_graph_attr', '_handle_deferred_dependencies', '_has_captured_ref', '_inputs', '_is_graph_tensor', '_list_extra_dependencies_for_serialization', '_list_functions_for_serialization', '_lookup_dependency', '_make_initializable_iterator', '_make_one_shot_it

In [5]:
train_data = train_data.__iter__()
print(next(train_data))

(<tf.Tensor: id=199, shape=(), dtype=string, numpy=b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.'>, <tf.Tensor: id=200, shape=(), dtype=int64, numpy=0>)


In [7]:
print(next(train_data))

(<tf.Tensor: id=201, shape=(), dtype=string, numpy=b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends. It did serve its purpose, at least until we were old enough and knowledgeable enough to analyze and create our own opinions. I was particularly terrified of what the newly-converted post-rapture Christians had to endure when not receiving the mark of the beast. I don't want to spoil the movie for those who haven't seen it so I will not mention details of the scenes, but I can still picture them in my head... and it's been 19 years.">, <tf.Tensor: id=202, shape=(), dtype=int64, numpy=0>)


In [23]:
training_sentences = []
training_labels = []
for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())


In [94]:
#training_sentences
training_sentences[1]

'b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven\'t seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends. It did serve its purpose, at least until we were old enough and knowledgeable enough to analyze and create our own opinions. I was particularly terrified of what the newly-converted post-rapture Christians had to endure when not receiving the mark of the beast. I don\'t want to spoil the movie for those who haven\'t seen it so I will not mention details of the scenes, but I can still picture them in my head... and it\'s been 19 years."'

In [95]:
#training_labels
training_labels[1]

0

In [45]:
type(training_labels)

list

In [18]:
testing_sentences = []
testing_labels = []
for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [30]:
training_labels_f = np.array(training_labels)
training_labels_f

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [31]:
testing_labels_f = np.array(testing_labels)
testing_labels_f

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [57]:
vocab_size = 10000
oov_tok = '<OOV>'
max_len = 120
trunc = 'post'
embedding_dim =16

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [36]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

In [39]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences , maxlen = max_len, truncating= trunc)

In [40]:
test_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(test_sequences , maxlen = max_len, truncating= trunc)

In [92]:
#word_index

In [91]:
reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])
#reverse_word_index

In [48]:
padded[1]

array([  59,  390,   87,   11,   14,   37,  185, 1377,    3,    1,   55,
         11,  216,   10,   12,   18,   14,    2, 1217,    5,  112,    1,
       3587,   16,   73,   11, 1449,  111,   10,  235,   11,   14,    9,
       8986, 1257,   32,    4,    1,  396,   37,   11,  240,   24,  250,
         51,  970,   10,   64,   28,   22,   73,  150,  190,   11,   81,
        136,   13,   10,  318,   35, 1404,   22,   73,    3,   92,    5,
         63,  373,   10,  123, 2935,   94, 1302,   32,  219,  374,   75,
         72,  154,  194,    3,    1,  194,    6, 8659,    3,  995,  262,
        203, 4754,   11,   14,  584, 5682,    5,   51,    2, 4755, 9330,
       1218, 9163, 4635,   71,    6, 4376,   55,   24, 5618,    2,  955,
          5,    2, 2751,   11,  174,  182,    6, 2392,    2,   18])

In [54]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [55]:
print(decode_review(padded[1]))

b perhaps because i was so young innocent and <OOV> when i saw it this movie was the cause of many <OOV> nights for me i haven't seen it since i was in seventh grade at a <OOV> school so i am not sure what effect it would have on me now however i will say that it left an impression on me and most of my friends it did serve its purpose at least until we were old enough and <OOV> enough to analyze and create our own opinions i was particularly terrified of what the newly converted post rapture christians had to endure when not receiving the mark of the beast i don't want to spoil the movie


In [56]:
training_sentences[1]

'b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven\'t seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends. It did serve its purpose, at least until we were old enough and knowledgeable enough to analyze and create our own opinions. I was particularly terrified of what the newly-converted post-rapture Christians had to endure when not receiving the mark of the beast. I don\'t want to spoil the movie for those who haven\'t seen it so I will not mention details of the scenes, but I can still picture them in my head... and it\'s been 19 years."'

In [59]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation ='relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics =['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [60]:
num_epochs = 10
model.fit(padded, training_labels_f, epochs = num_epochs, validation_data=(test_padded, testing_labels_f))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - ETA: 9:10 - loss: 0.6910 - accuracy: 0.56 - ETA: 36s - loss: 0.6918 - accuracy: 0.5254 - ETA: 18s - loss: 0.6928 - accuracy: 0.514 - ETA: 12s - loss: 0.6920 - accuracy: 0.517 - ETA: 9s - loss: 0.6919 - accuracy: 0.518 - ETA: 8s - loss: 0.6919 - accuracy: 0.51 - ETA: 7s - loss: 0.6914 - accuracy: 0.51 - ETA: 6s - loss: 0.6906 - accuracy: 0.51 - ETA: 5s - loss: 0.6898 - accuracy: 0.52 - ETA: 5s - loss: 0.6879 - accuracy: 0.52 - ETA: 4s - loss: 0.6871 - accuracy: 0.52 - ETA: 4s - loss: 0.6845 - accuracy: 0.54 - ETA: 3s - loss: 0.6810 - accuracy: 0.55 - ETA: 3s - loss: 0.6760 - accuracy: 0.56 - ETA: 3s - loss: 0.6714 - accuracy: 0.57 - ETA: 3s - loss: 0.6658 - accuracy: 0.58 - ETA: 2s - loss: 0.6593 - accuracy: 0.59 - ETA: 2s - loss: 0.6528 - accuracy: 0.60 - ETA: 2s - loss: 0.6454 - accuracy: 0.61 - ETA: 2s - loss: 0.6386 - accuracy: 0.62 - ETA: 2s - loss: 0.6308 - accuracy: 0.63 - 

25000/25000 [==============================] - ETA: 10s - loss: 0.0012 - accuracy: 1.000 - ETA: 3s - loss: 0.0019 - accuracy: 1.000 - ETA: 3s - loss: 0.0018 - accuracy: 1.00 - ETA: 3s - loss: 0.0018 - accuracy: 1.00 - ETA: 2s - loss: 0.0018 - accuracy: 1.00 - ETA: 2s - loss: 0.0018 - accuracy: 1.00 - ETA: 2s - loss: 0.0018 - accuracy: 1.00 - ETA: 2s - loss: 0.0017 - accuracy: 1.00 - ETA: 2s - loss: 0.0017 - accuracy: 1.00 - ETA: 2s - loss: 0.0021 - accuracy: 0.99 - ETA: 2s - loss: 0.0021 - accuracy: 0.99 - ETA: 2s - loss: 0.0021 - accuracy: 0.99 - ETA: 2s - loss: 0.0020 - accuracy: 0.99 - ETA: 2s - loss: 0.0020 - accuracy: 0.99 - ETA: 2s - loss: 0.0019 - accuracy: 0.99 - ETA: 2s - loss: 0.0019 - accuracy: 0.99 - ETA: 2s - loss: 0.0019 - accuracy: 0.99 - ETA: 2s - loss: 0.0019 - accuracy: 0.99 - ETA: 2s - loss: 0.0018 - accuracy: 0.99 - ETA: 2s - loss: 0.0018 - accuracy: 0.99 - ETA: 1s - loss: 0.0018 - accuracy: 0.99 - ETA: 1s - loss: 0.0018 - accuracy: 0.99 - ETA: 1s - loss: 0.0018 - a

In [67]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) #shape: (vocab_size, embedding_dim)

(10000, 16)


In [79]:
weights[1]

array([-0.00136961,  0.0850051 ,  0.0471346 ,  0.02368701,  0.08936676,
        0.01465387, -0.04302087, -0.13366103, -0.05391143,  0.02730086,
        0.05496754, -0.01043492, -0.03214395, -0.04211498, -0.14496863,
       -0.0421529 ], dtype=float32)

In [80]:
import io

In [81]:
out_v = io.open('vector.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
    word = reverse_word_index[word_num]
    embedding = weights[word_num]
    out_m.write(word + '\n')
    out_v.write('\t'.join([str(x) for x in embedding]) + '\n')
out_m.close()
out_v.close()